# Autonomous Driving Object Detection

## Setup

In [ ]:
# IMPORTS
import torch
import torchvision
import numpy as np
import os
import yaml

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# Used to solve the error "Error #15: Initializing libiomp5md.dll, but found libiomp5md.dll already initialized."
# This is not a recommended solution, but it is easy to implement.
# This problem appear to be present only when using torch and PIL inside anaconda.
# In colab the problem is not present. Also, maybe in future try to use the builtin python envs instead of anaconda.
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
# Check if GPU is available and set device
device = "cpu"
if torch.cuda.is_available:
  print("GPU AVAILABLE")
  device = torch.device("cuda:0")
else:
  print("GPU NOT AVAILABLE")

In [ ]:
# Set random seed
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

## Download Dataset

In [ ]:
# Mio url unico
!wget -O dataset.zip https://public.roboflow.com/ds/OSbfqB4WlB?key=4L6u3xfkPg

In [ ]:
# Unzip the dataset (capture is used to suppress the output of the colab cell)

%%capture
!unzip dataset.zip -d dataset

In [ ]:
from data_loading.dir_utils import print_directory_tree, print_file_from_dir

# Print directory tree
print_directory_tree("dataset")

In [ ]:
print_file_from_dir(os.path.join("dataset","export","images"), file_n=43)

In [ ]:
# Note: annotations for bounding boxes are in the form:
# class_id center_x center_y width height
# All values are normalized

print_file_from_dir(os.path.join("dataset","export","labels"), file_n=100)

## Rearrange dataset in folders

In [ ]:
image_ext = ".jpg"
label_ext = ".txt"
image_width = 512
image_height = 512

In [ ]:
from data_loading.split_dataset import split_dataset

# Separate the samples in 3 sets creating 3 directories: train, validation, test
# In each directory copy a random subset of the total data available, without repetitions and with fixed proportions.
# At the end we will have the 3 directories each one containing two directories images and labels
split_dataset(os.path.join("dataset","export","images"),
              image_ext,
              os.path.join("dataset","export","labels"),
              label_ext,
              os.path.join("dataset","splitted"))

In [ ]:
print_directory_tree(os.path.join("dataset","splitted"))

## Create Dataloaders

In [ ]:
# Read YAML file to obtain the list of objects classes
with open(os.path.join("dataset","data.yaml"), 'r') as yaml_file:
    yaml_obj = yaml.safe_load(yaml_file)
    classes = yaml_obj["names"]
    print(classes)

In [ ]:
from data_loading.autonomous_driving_dataset import AutonomousDrivingDataset

# Create datasets

adod_transforms = {"train": torchvision.transforms.Compose([torchvision.transforms.ToTensor()]),
                  "val": torchvision.transforms.ToTensor()}


train_dataset = AutonomousDrivingDataset(os.path.join("dataset", "splitted", "train", "images"),
                                         image_ext,
                                         image_width,
                                         image_height,
                                         os.path.join("dataset","splitted","train","labels"),
                                         label_ext,
                                         classes,
                                         adod_transforms["train"])

validation_dataset = AutonomousDrivingDataset(os.path.join("dataset", "splitted", "validation", "images"),
                                              image_ext,
                                              image_width,
                                              image_height,
                                              os.path.join("dataset","splitted","validation","labels"),
                                              label_ext,
                                              classes,
                                              adod_transforms["val"])

test_dataset = AutonomousDrivingDataset(os.path.join("dataset", "splitted", "test", "images"),
                                        image_ext,
                                        image_width,
                                        image_height,
                                        os.path.join("dataset","splitted","test","labels"),
                                        label_ext,
                                        classes,
                                        adod_transforms["val"])

In [ ]:
print(train_dataset[3])

In [ ]:
from data_loading.dl_utils import collate_fn

# Create dataloaders with datasets

num_workers = 0 #4
size_batch = 8 #8

train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                              batch_size=size_batch, 
                                              shuffle=True, 
                                              pin_memory=True, 
                                              num_workers=num_workers,
                                              collate_fn=collate_fn)

validation_dataloader = torch.utils.data.DataLoader(validation_dataset,
                                                    batch_size=size_batch,
                                                    shuffle=False,
                                                    num_workers=num_workers,
                                                    collate_fn=collate_fn)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=size_batch,
                                              shuffle=False,
                                              num_workers=num_workers,
                                              collate_fn=collate_fn)

In [ ]:
# Try the dataloader
it = iter(train_dataloader)
first = next(it)
print(first)

## Define the neural model

In [ ]:
import torchvision.models.detection.backbone_utils as ut
from torchvision.ops.feature_pyramid_network import LastLevelP6P7
from torchvision.models.detection import RetinaNet
from torchvision.models.detection.anchor_utils import AnchorGenerator


use_big_model = False

# Instantiate the model

if use_big_model:
    # The model builders retinanet_resnet50_fpn and retinanet_resnet50_fpn_v2 can be used to instantiate a
    # RetinaNet model, with or without pre-trained weights.
    # All the model builders internally rely on the torchvision.models.detection.retinanet.RetinaNet base class. 
    #model = torchvision.models.detection.retinanet_resnet50_fpn(weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT)
    model = torchvision.models.detection.retinanet_resnet50_fpn()
else:
    # https://github.com/pytorch/vision/blob/master/torchvision/models/detection/backbone_utils.py#L49
    backbone = ut.resnet_fpn_backbone('resnet18', 
                                      pretrained=False)

    anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=((0.5, 1.0, 2.0)))

    model = RetinaNet(backbone,
                      len(train_dataset.classes),
                      anchor_generator=anchor_generator)

model.to(device)
    
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Number of trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

## Try the model before training it

In [ ]:
from object_detection.object_detection_utils import detect_objects, draw_bounding_boxes

In [ ]:
x = validation_dataset[3][0].to(device)

In [ ]:
plt.imshow(x.cpu().permute(1, 2, 0))

In [ ]:
print(detect_objects(x, model, 0.1, classes))

## Model Training

In [ ]:
from training.train import train_one_epoch

In [ ]:
starting_lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=starting_lr)

In [ ]:
n_epochs = 1
for i in range(1, n_epochs+1):
    train_one_epoch(model, train_dataloader, optimizer, device)
    #TODO: implement also validation with the validation dataset

## Model Testing

In [ ]:
#TODO: Test the model with the test dataset

In [ ]:
# Test the model on a single image
x = validation_dataset[3][0].to(device)

In [ ]:
boxes_filtered, scores_filtered, labels_filtered, categories_filtered = detect_objects(x, model, 0.1, classes)
print(categories_filtered)

In [ ]:
image = torchvision.transforms.ToPILImage()(x)
draw_bounding_boxes(image, boxes_filtered, classes, labels_filtered,
                    scores_filtered, colors=[(255,0,0)]*20, normalized_coordinates=False, add_text=True)